In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Load the datasets
#eyetracking_data = pd.read_csv("D:\MasterThesis\Experiment_1\Complete_Data\Complete_Data_AngularV.csv")
#pointing_data = pd.read_csv("D:\MasterThesis\Experiment_1\PointingTask\PointingTask_Integrated_exp1.csv")

In [9]:
eyetracking_data_365.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117117 entries, 0 to 117116
Data columns (total 70 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   Unnamed: 0                     117117 non-null  int64         
 1   index                          117117 non-null  int64         
 2   SubjectID                      117117 non-null  int64         
 3   Eyetracking                    117117 non-null  int64         
 4   timeStampDataPointStart        117117 non-null  datetime64[ns]
 5   timeStampDataPointEnd          117117 non-null  datetime64[ns]
 6   timeStampGetVerboseData        117117 non-null  float64       
 7   combinedGazeValidityBitmask    117117 non-null  int64         
 8   rayCastHitsCombinedEyes        117117 non-null  object        
 9   eyePositionCombinedWorld.x     117117 non-null  float64       
 10  eyePositionCombinedWorld.y     113150 non-null  float64       
 11  

In [10]:
aligned_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111811 entries, 0 to 111810
Data columns (total 72 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   Unnamed: 0                     111811 non-null  int64         
 1   index                          111811 non-null  int64         
 2   SubjectID                      111811 non-null  int64         
 3   Eyetracking                    111811 non-null  int64         
 4   timeStampDataPointStart        111811 non-null  datetime64[ns]
 5   timeStampDataPointEnd          111811 non-null  datetime64[ns]
 6   timeStampGetVerboseData        111811 non-null  float64       
 7   combinedGazeValidityBitmask    111811 non-null  int64         
 8   rayCastHitsCombinedEyes        111811 non-null  object        
 9   eyePositionCombinedWorld.x     111811 non-null  float64       
 10  eyePositionCombinedWorld.y     107876 non-null  float64       
 11  

In [7]:
import pandas as pd

# Load your dataframes and filter for Subject 365
eyetracking_data_365 =  pd.read_csv("/Users/tracysanchezpacheco/Desktop/eyetracking_365.csv")
pointing_data_365 = pd.read_csv("/Users/tracysanchezpacheco/Desktop/pointing_data_365.csv")

# Convert Unix timestamps to datetime format
eyetracking_data_365['timeStampDataPointStart'] = pd.to_datetime(eyetracking_data_365['timeStampDataPointStart'], unit='s')
eyetracking_data_365['timeStampDataPointEnd'] = pd.to_datetime(eyetracking_data_365['timeStampDataPointEnd'], unit='s')
pointing_data_365['TimeStampBegin'] = pd.to_datetime(pointing_data_365['TimeStampBegin'])
pointing_data_365['TimeStampEnd'] = pd.to_datetime(pointing_data_365['TimeStampEnd'])

# Calculate relative duration once
pointing_duration_total = (pointing_data_365['TimeStampEnd'].max() - pointing_data_365['TimeStampBegin'].min()).total_seconds()

aligned_data = []

for _, pointing_row in pointing_data_365.iterrows():
    subject_id = pointing_row['SubjectID']
    trial_number = pointing_row['TrialNumber']
    pointing_start = pointing_row['TimeStampBegin']
    pointing_end = pointing_row['TimeStampEnd']
    pointing_location = pointing_row['PointingTaskStartingLocations']
    
    # Calculate the relative duration within the pointing experiment
    pointing_duration = (pointing_end - pointing_start).total_seconds()
    if pointing_duration_total == 0:
        relative_duration = 0
    else:
        relative_duration = (pointing_start - pointing_data_365['TimeStampBegin'].min()).total_seconds() / pointing_duration_total
        relative_duration = max(0, min(relative_duration, 1))  # Clamp between 0 and 1
    
    # Calculate the time offset based on the reference
    time_offset = relative_duration * reference_diff
    
    # Calculate the corresponding eyetracking timestamps
    eyetracking_start = reference_start + pd.Timedelta(seconds=time_offset)
    eyetracking_end = eyetracking_start + pd.Timedelta(seconds=pointing_duration)
    
    # Filter relevant columns from eyetracking_data using boolean indexing
    eyetracking_data_subset = eyetracking_data_365.loc[(eyetracking_data_365['SubjectID'] == subject_id) &
                                                       (eyetracking_data_365['timeStampDataPointStart'] >= eyetracking_start) &
                                                       (eyetracking_data_365['timeStampDataPointEnd'] <= eyetracking_end)].copy()
    
    # Add pointing trial info to eyetracking_data_subset
    eyetracking_data_subset['TrialNumber'] = trial_number
    eyetracking_data_subset['PointingTaskStartingLocations'] = pointing_location
    
    # Append to the aligned_data list
    aligned_data.append(eyetracking_data_subset)

# Concatenate the aligned dataframes
aligned_data_df = pd.concat(aligned_data, ignore_index=True)

# Print the first few rows of the aligned dataframe
aligned_data_df.head()


,Unnamed: 0,index,SubjectID,Eyetracking,timeStampDataPointStart,timeStampDataPointEnd,timeStampGetVerboseData,combinedGazeValidityBitmask,rayCastHitsCombinedEyes,eyePositionCombinedWorld.x,...,isFix,corrected_vel,events,length,distance,avg_dist,names,Collider_CategoricalN,TrialNumber,PointingTaskStartingLocations
0,0,0,365,1,2021-11-18 10:22:22.621986304,2021-11-18 10:22:22.624962048,1.637231e+09,0,[{'hitPointOnObject': {'x': -116.4806289672851...,-114.327370,...,NaN,NaN,NaN,NaN,NaN,NaN,pavement_K.002,Background,1,21
1,1,1,365,1,2021-11-18 10:22:22.633890560,2021-11-18 10:22:22.636866304,1.637231e+09,0,[{'hitPointOnObject': {'x': -116.4808349609375...,-114.327370,...,0.000000,0.000000,2.0,0.513359,143.589519,143.571742,pavement_K.002,Background,1,21
2,2,2,365,1,2021-11-18 10:22:22.650754560,2021-11-18 10:22:22.653731072,1.637231e+09,3,"[{'hitPointOnObject': {'x': 63.72035217285156,...",-114.327370,...,0.000000,0.000000,NaN,0.513359,143.589519,143.571742,pavement_K.002,Background,1,21
3,3,3,365,1,2021-11-18 10:22:23.113523200,2021-11-18 10:22:23.116993792,1.637231e+09,3,"[{'hitPointOnObject': {'x': 23.45851707458496,...",-114.294662,...,0.106728,0.106728,NaN,0.513359,143.560421,143.571742,pavement_K.002,Background,1,21
4,4,4,365,1,2021-11-18 10:22:23.125426944,2021-11-18 10:22:23.128401408,1.637231e+09,3,"[{'hitPointOnObject': {'x': 6.599761009216309,...",-114.294388,...,4.581290,4.581290,NaN,0.513359,143.560125,143.571742,pavement_K.002,Background,1,21


In [17]:
# Load your dataframes and filter for Subject 365
eyetracking_data_365 =  pd.read_csv("/Users/tracysanchezpacheco/Desktop/eyetracking_365.csv")
pointing_data_365 = pd.read_csv("/Users/tracysanchezpacheco/Desktop/pointing_data_365.csv")

# Convert Unix timestamps to datetime format
eyetracking_data_365['timeStampDataPointStart'] = pd.to_datetime(eyetracking_data_365['timeStampDataPointStart'], unit='s')
eyetracking_data_365['timeStampDataPointEnd'] = pd.to_datetime(eyetracking_data_365['timeStampDataPointEnd'], unit='s')
pointing_data_365['TimeStampBegin'] = pd.to_datetime(pointing_data_365['TimeStampBegin'])
pointing_data_365['TimeStampEnd'] = pd.to_datetime(pointing_data_365['TimeStampEnd'])

aligned_data = []

# Set the reference timestamps based on eyetracking data
reference_start = eyetracking_data_365['timeStampDataPointStart'].min()
reference_end = eyetracking_data_365['timeStampDataPointEnd'].max()
reference_diff = (reference_end - reference_start).total_seconds()

for index, pointing_row in pointing_data_365.iterrows():
    subject_id = pointing_row['SubjectID']
    trial_number = pointing_row['TrialNumber']
    pointing_start = pointing_row['TimeStampBegin']
    pointing_end = pointing_row['TimeStampEnd']
    pointing_location = pointing_row['PointingTaskStartingLocations']
    
    # Calculate the relative duration within the pointing experiment
    pointing_duration = (pointing_end - pointing_start).total_seconds()
    pointing_duration_total = (pointing_data_365['TimeStampEnd'].max() - pointing_data_365['TimeStampBegin'].min()).total_seconds()
    if pointing_duration_total == 0:
        relative_duration = 0
    else:
        relative_duration = (pointing_start - pointing_data_365['TimeStampBegin'].min()).total_seconds() / pointing_duration_total
        relative_duration = max(0, min(relative_duration, 1))  # Clamp between 0 and 1
    
    # Calculate the time offset based on the reference
    time_offset = relative_duration * reference_diff
    
    # Calculate the corresponding eyetracking timestamps
    eyetracking_start = reference_start + pd.Timedelta(seconds=time_offset)
    eyetracking_end = eyetracking_start + pd.Timedelta(seconds=pointing_duration)
    
    # Filter relevant columns from eyetracking_data
    eyetracking_data_subset = eyetracking_data_365.loc[(eyetracking_data_365['SubjectID'] == subject_id) &
                                                    (eyetracking_data_365['timeStampDataPointStart'] >= eyetracking_start) &
                                                    (eyetracking_data_365['timeStampDataPointEnd'] <= eyetracking_end)].copy()
    
    # Add pointing trial info to eyetracking_data_subset
    eyetracking_data_subset['TrialNumber'] = trial_number
    eyetracking_data_subset['PointingTaskStartingLocations'] = pointing_location
    
    # Append to the aligned_data list
    aligned_data.append(eyetracking_data_subset)

# Concatenate the aligned dataframes
aligned_data_df = pd.concat(aligned_data, ignore_index=True)

# Print the results
aligned_data_df.head()


,Unnamed: 0,index,SubjectID,Eyetracking,timeStampDataPointStart,timeStampDataPointEnd,timeStampGetVerboseData,combinedGazeValidityBitmask,rayCastHitsCombinedEyes,eyePositionCombinedWorld.x,...,isFix,corrected_vel,events,length,distance,avg_dist,names,Collider_CategoricalN,TrialNumber,PointingTaskStartingLocations
0,0,0,365,1,2021-11-18 10:22:22.621986304,2021-11-18 10:22:22.624962048,1.637231e+09,0,[{'hitPointOnObject': {'x': -116.4806289672851...,-114.327370,...,NaN,NaN,NaN,NaN,NaN,NaN,pavement_K.002,Background,1,21
1,1,1,365,1,2021-11-18 10:22:22.633890560,2021-11-18 10:22:22.636866304,1.637231e+09,0,[{'hitPointOnObject': {'x': -116.4808349609375...,-114.327370,...,0.000000,0.000000,2.0,0.513359,143.589519,143.571742,pavement_K.002,Background,1,21
2,2,2,365,1,2021-11-18 10:22:22.650754560,2021-11-18 10:22:22.653731072,1.637231e+09,3,"[{'hitPointOnObject': {'x': 63.72035217285156,...",-114.327370,...,0.000000,0.000000,NaN,0.513359,143.589519,143.571742,pavement_K.002,Background,1,21
3,3,3,365,1,2021-11-18 10:22:23.113523200,2021-11-18 10:22:23.116993792,1.637231e+09,3,"[{'hitPointOnObject': {'x': 23.45851707458496,...",-114.294662,...,0.106728,0.106728,NaN,0.513359,143.560421,143.571742,pavement_K.002,Background,1,21
4,4,4,365,1,2021-11-18 10:22:23.125426944,2021-11-18 10:22:23.128401408,1.637231e+09,3,"[{'hitPointOnObject': {'x': 6.599761009216309,...",-114.294388,...,4.581290,4.581290,NaN,0.513359,143.560125,143.571742,pavement_K.002,Background,1,21


In [18]:
aligned_data_df.TrialNumber.unique()

array([  1,  78,  88,  95, 131, 137, 146, 180, 289, 292,   2,  21, 145,
       242, 245,   3, 140, 147, 183, 204, 291,   4,  35,  51,  91, 172,
       248,   5, 105, 132, 141,   6, 104, 133, 150, 279, 285,   7,  16,
        96, 123, 148, 176, 287, 316,   8,  11,  63,  83, 165, 196, 318,
         9,  47, 190,  10,  54, 113, 154, 216, 220, 265,  12,  50,  65,
       119, 129, 173, 268, 269, 283,  13,  46, 198, 213, 261, 302, 312,
        14,  87, 162, 174, 184, 304, 315,  15,  44, 111, 189, 229, 317,
        17,  28,  97, 199, 234, 237, 278, 284, 293,  18,  53,  68,  92,
       106, 335,  19,  99, 160, 288, 301,  20,  98, 127, 200, 224, 233,
       243, 250, 256,  22,  38,  64, 187,  23, 177, 182, 207, 222,  24,
        66, 115, 121, 157, 221, 319, 320,  25,  41,  42, 130, 186, 206,
       296, 309,  26,  62,  89, 122, 138, 181, 215, 219, 255, 274,  27,
        59, 114, 116, 306, 329, 331,  29, 102, 139, 155, 223, 225, 238,
       240, 277, 303, 326,  30,  36,  94, 109, 195, 254, 276, 29

In [12]:
aligned_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111811 entries, 0 to 111810
Data columns (total 72 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   Unnamed: 0                     111811 non-null  int64         
 1   index                          111811 non-null  int64         
 2   SubjectID                      111811 non-null  int64         
 3   Eyetracking                    111811 non-null  int64         
 4   timeStampDataPointStart        111811 non-null  datetime64[ns]
 5   timeStampDataPointEnd          111811 non-null  datetime64[ns]
 6   timeStampGetVerboseData        111811 non-null  float64       
 7   combinedGazeValidityBitmask    111811 non-null  int64         
 8   rayCastHitsCombinedEyes        111811 non-null  object        
 9   eyePositionCombinedWorld.x     111811 non-null  float64       
 10  eyePositionCombinedWorld.y     107876 non-null  float64       
 11  

In [4]:
# Define the file path
csv_file_path = r'D:\MasterThesis\Experiment_1\Matching\aligned_data.csv'

# Save the dataframe to CSV
aligned_data_df.to_csv(csv_file_path, index=False)

print(f"Dataframe saved as {csv_file_path}")


Dataframe saved as D:\MasterThesis\Experiment_1\Matching\aligned_data.csv
